In [4]:
import os
import numpy as np
import random
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense

In [27]:
import re
import nltk
import string
#nltk.download('punkt')
nltk.download('wordnet') 
nltk.download('stopwords')
nltk.download('omw-1.4')

from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ajaykarthicksenthilkumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ajaykarthicksenthilkumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ajaykarthicksenthilkumar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
bio_files_dir = '../../data/bio_data_files'

In [12]:
all_files = os.listdir(bio_files_dir)

In [13]:
bio_files = [f for f in all_files if f.endswith('.bio')]

In [ ]:
wn = nltk.WordNetLemmatizer()    
    text = wn.lemmatize(text, 'r')

In [47]:
def clean_text(text):
    text = text.lower()
    text = "".join([c for c in text if c not in string.punctuation])
    wn = nltk.WordNetLemmatizer()    
    text = wn.lemmatize(text, 'r')
    return text

In [15]:
MAX_LENGTH=110
TEST_SIZE=0.2

In [16]:
for file in bio_files:
    with open(os.path.join(bio_files_dir, file), 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip() == '':
                continue
            word, tag = line.strip().split('\t')
            word = clean_text(word)
            if word.strip():
                print(word, tag)
            elif tag != 'O':
                print(line)
                print('Alert')


a O
36 B-AGE
yr I-AGE
old I-AGE
previously B-HIS
healthy I-HIS
sri B-PER
lankan I-PER
male B-SEX
who O
takes B-OCC
care I-OCC
of I-OCC
a I-OCC
horse I-OCC
presented B-CLE
to O
the O
medical B-NBL
casualty I-NBL
ward I-NBL
with O
fever B-SIG
arthralgia I-SIG
and O
myalgia B-SIG
for O
one B-DUR
day I-DUR
he O
complained O
of O
mild B-SEV
dysuria B-SIG
but O
had O
normal B-LAB
urine B-DIA
output I-DIA
he O
did O
not O
have O
chest B-BST
pain B-SIG
or O
shortness B-SIG
of I-SIG
breath I-SIG
far O
inquiry O
revealed O
that O
he O
was O
treated B-HIS
for I-HIS
leptospirosis I-HIS
during I-HIS
a I-HIS
febrile I-HIS
illness I-HIS
in I-HIS
the I-HIS
past I-HIS
on O
examination B-DIA
patient O
was O
afebrile B-SIG
anicteric I-SIG
his O
blood B-DIA
pressure I-DIA
was O
9060 B-LAB
mmhg I-LAB
and O
pulse B-DIA
rate I-DIA
76 B-LAB
bpm I-LAB
rest O
of O
the O
examination B-DIA
was O
unremarkable B-LAB
initial O
investigations B-DIA
performed O
on O
the O
day O
of O
admission O
revealed O
neutrophil B

the O
patient O
is O
a O
59 B-AGE
year I-AGE
old I-AGE
man B-SEX
diagnosed O
with O
pd B-DIS
at O
the O
age B-DAT
of I-DAT
49 I-DAT
years I-DAT
fluctuations B-SIG
with O
recurring O
“off B-OTE
states I-OTE
and O
peak B-DET
dose I-DET
dyskinesias B-DIS
had O
severely O
diminished B-LAB
the O
patient’s O
quality B-DIA
of I-DIA
life I-DIA
thus O
the O
patient O
was O
deemed O
a O
candidate O
for O
bilateral B-DET
subthalamic I-DET
dbs B-THP
multidisciplinary B-DIA
evaluation I-DIA
with O
75 B-LAB
positive I-LAB
response O
in O
the O
standardized B-DIA
levodopa I-DIA
test I-DIA
exclusion O
of O
cognitive B-DIS
decline I-DIS
or O
psychiatric B-DIS
comorbitity I-DIS
and O
absence O
of O
structural O
brain B-DIS
damage I-DIS
potentially O
relevant O
to O
the O
lead O
placement O
led O
to O
the O
unanimous O
decision O
for O
dbs B-THP
placement O
when O
decision O
was O
made O
to O
proceed O
to O
surgery B-THP
the O
medication B-MED
consisted O
of O
11125 B-DOS
mg I-DOS
of O
levodopa B-MED
and

In [9]:

def load_data(data_dir, max_length=None):

    # Load all files in the data directory
    all_files = os.listdir(data_dir)

    # Filter only the files with the .bio extension
    bio_files = [f for f in all_files if f.endswith('.bio')]

    # Initialize lists to hold sentences and labels
    sentences = []
    labels = []

    # Loop through each file and read the sentences and labels
    for file in bio_files:
        with open(os.path.join(data_dir, file), 'r', encoding='utf-8') as f:
            current_sentences = []
            current_labels = []
            for line in f:
                if line.strip() == '':
                    
                    # If we encounter a blank line, it means we've reached the end of a sentence
                    if len(current_sentences) > 0:
                        
                        # Add the current sentence and labels to the list
                        sentences.append(current_sentences)
                        labels.append(current_labels)
                        # Reset the current sentence and labels lists
                        current_sentences = []
                        current_labels = []
                else:
                    
                    # Otherwise, split the line into its word and label components
                    word, label = line.strip().split('\t')
                    word = clean_text(word)
                    if word.strip():
                        current_sentences.append(word)
                        current_labels.append(label)

    # Shuffle the sentences and labels
    combined = list(zip(sentences, labels))
    random.shuffle(combined)
    sentences[:], labels[:] = zip(*combined)

    # Split the data into training, validation, and test sets

    num_sentences = len(sentences)
    num_train = int(num_sentences * (1 - TEST_SIZE - 0.1))
    num_valid = int(num_sentences * 0.1)

    train_sentences = sentences[:num_train]
    train_labels = labels[:num_train]

    valid_sentences = sentences[num_train:num_train+num_valid]
    valid_labels = labels[num_train:num_train+num_valid]

    test_sentences = sentences[num_train+num_valid:]
    test_labels = labels[num_train+num_valid:]

    # Convert the labels to one-hot encoding
    unique_labels = set(element for sublist in labels for element in sublist)
    label_to_index = {label: id+1 for id, label in enumerate(sorted(unique_labels))}
    index_to_label = {id: label for label, id in label_to_index.items()}

    # Add the new label and ID to the dictionaries
    label_to_index['<PAD>'] = 0
    index_to_label[0] = '<PAD>'

    num_classes = len(index_to_label) - 1

    train_labels = [[label_to_index[label] for label in labels] for labels in train_labels]
    train_labels = pad_sequences(train_labels, maxlen=max_length, padding='post', value=num_classes)
    train_labels = to_categorical(train_labels, num_classes=num_classes+1)

    valid_labels = [[label_to_index[label] for label in labels] for labels in valid_labels]
    valid_labels = pad_sequences(valid_labels, maxlen=max_length, padding='post', value=num_classes)
    valid_labels = to_categorical(valid_labels, num_classes=num_classes+1)

    test_labels = [[label_to_index[label] for label in labels] for labels in test_labels]
    test_labels = pad_sequences(test_labels, maxlen=max_length, padding='post', value=num_classes)
    test_labels = to_categorical(test_labels, num_classes=num_classes+1)

    return (train_sentences, train_labels), (valid_sentences, valid_labels), (test_sentences, test_labels), label_to_index, index_to_label

In [10]:
(train_sentences, train_labels), (val_sentences, val_labels), (test_sentences, test_labels), label2id, id2label = load_data(bio_files_dir, MAX_LENGTH)

In [11]:
len(train_sentences), train_labels.shape

(3038, (3038, 110, 201))

[['the',
  'full',
  'blood',
  'count',
  'showed',
  'a',
  'normal',
  'haemoglobin',
  'and',
  'white',
  'cell',
  'count'],
 ['there',
  'were',
  'numerous',
  'mitotic',
  'figures',
  'ranging',
  'from',
  '30',
  'to',
  '60',
  'per',
  '10',
  'high',
  'power',
  'fields',
  'and',
  'some',
  'of',
  'them',
  'were',
  'abnormal'],
 ['no',
  'cranial',
  'nerve',
  'deficits',
  'were',
  'found',
  'and',
  'swallowing',
  'was',
  'normal'],
 ['his',
  'complicated',
  'medical',
  'history',
  'included',
  'pancreatic',
  'insufficiency',
  'liver',
  'transplantation',
  'in',
  '2004',
  'steroid',
  'induced',
  'diabetes',
  'end',
  'stage',
  'renal',
  'disease',
  'and',
  'testicular',
  'cancer'],
 ['however',
  'due',
  'to',
  'the',
  'high',
  'surgical',
  'risk',
  'of',
  'bypass',
  'endovascular',
  'treatment',
  'was',
  'suggested'],
 ['mitoses',
  'were',
  'numerous',
  'and',
  'structurally',
  'abnormal',
  'forms',
  'were',
  'common'],

In [12]:
# Convert the input sentences to sequences of word indices
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sentences)

train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

# Pad the sequences to a fixed length
train_sequences_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')
val_sequences_padded = pad_sequences(val_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')
test_sequences_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

In [13]:
id2label

{1: 'B-Activity',
 2: 'B-Activity;B-History',
 3: 'B-Activity;B-Therapeutic_procedure',
 4: 'B-Activity;I-History',
 5: 'B-Administration',
 6: 'B-Administration;I-Dosage',
 7: 'B-Age',
 8: 'B-Age;B-Date',
 9: 'B-Age;I-Date',
 10: 'B-Age;I-Family_history',
 11: 'B-Area',
 12: 'B-Biological_attribute',
 13: 'B-Biological_structure',
 14: 'B-Biological_structure;B-History',
 15: 'B-Biological_structure;B-Sign_symptom',
 16: 'B-Biological_structure;I-Detailed_description',
 17: 'B-Biological_structure;I-History',
 18: 'B-Biological_structure;I-Sign_symptom',
 19: 'B-Clinical_event',
 20: 'B-Clinical_event;I-History',
 21: 'B-Color',
 22: 'B-Coreference',
 23: 'B-Coreference;I-Detailed_description',
 24: 'B-Date',
 25: 'B-Date;B-History',
 26: 'B-Date;I-History',
 27: 'B-Detailed_description',
 28: 'B-Detailed_description;B-Disease_disorder',
 29: 'B-Detailed_description;B-Family_history',
 30: 'B-Detailed_description;B-History',
 31: 'B-Detailed_description;I-Family_history',
 32: 'B-Deta

In [14]:
NUM_CLASSES = len(id2label)

In [15]:
# Define the model architecture
model = tf.keras.models.Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=216, input_length=MAX_LENGTH),
    Bidirectional(LSTM(units=64, return_sequences=True)),
    Dense(NUM_CLASSES, activation='softmax')
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 110, 216)          1715904   
                                                                 
 bidirectional (Bidirectiona  (None, 110, 128)         143872    
 l)                                                              
                                                                 
 dense (Dense)               (None, 110, 201)          25929     
                                                                 
Total params: 1,885,705
Trainable params: 1,885,705
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the model
model.fit(train_sequences_padded, train_labels, epochs=20, validation_data=(val_sequences_padded, val_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_sequences_padded, test_labels)

# Print the test accuracy
print('Test accuracy:', test_acc)

Epoch 1/20


2023-04-05 21:10:24.105453: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


95/95 [==============================] - 8s 73ms/step - loss: 1.1454 - accuracy: 0.8908 - val_loss: 0.4588 - val_accuracy: 0.9091
Epoch 2/20
95/95 [==============================] - 7s 75ms/step - loss: 0.4361 - accuracy: 0.9098 - val_loss: 0.4399 - val_accuracy: 0.9091
Epoch 3/20
95/95 [==============================] - 8s 80ms/step - loss: 0.4148 - accuracy: 0.9098 - val_loss: 0.4313 - val_accuracy: 0.9091
Epoch 4/20
95/95 [==============================] - 7s 72ms/step - loss: 0.3858 - accuracy: 0.9099 - val_loss: 0.4263 - val_accuracy: 0.9091
Epoch 5/20
95/95 [==============================] - 7s 73ms/step - loss: 0.3557 - accuracy: 0.9128 - val_loss: 0.4304 - val_accuracy: 0.9100
Epoch 6/20
95/95 [==============================] - 7s 73ms/step - loss: 0.3338 - accuracy: 0.9195 - val_loss: 0.4313 - val_accuracy: 0.9122
Epoch 7/20
95/95 [==============================] - 7s 76ms/step - loss: 0.3038 - accuracy: 0.9280 - val_loss: 0.4111 - val_accuracy: 0.9143
Epoch 8/20
95/95 [======

In [19]:
import re
import string
def tokenize_text(text):
    # Tokenize the text into a list of words
    tokens = []
    for sentence in text.split('\n'):
        for word in sentence.split():
            word = clean_text(word)
            if word.strip():
                tokens.append(word)
    return tokens

def predict(text):
    # tokens = re.findall(r'\b\w+\b', text)
    tokens = tokenize_text(text)
    sequence = tokenizer.texts_to_sequences([' '.join(token for token in tokens)])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post')

    # Make the prediction
    prediction = model.predict(np.array(padded_sequence))

    # Decode the prediction
    predicted_labels = np.argmax(prediction, axis=-1)
    predicted_labels = [id2label[i] for i in predicted_labels[0]]

    # Print the predicted named entities
    print("Predicted Named Entities:")
    for i in range(len(tokens)):
        print(f"{tokens[i]}: {predicted_labels[i]}")

In [20]:
predict("Leptospira IgM antibody performed on the 5th day of illness was positive (IgG negative).")

1/1 [==============================] - 0s 333ms/step
Predicted Named Entities:
leptospira: B-Detailed_description
igm: I-Detailed_description
antibody: I-Diagnostic_procedure
performed: O
on: O
the: O
5th: B-Date
day: I-Date
of: O
illness: O
was: O
positive: B-Lab_value
igg: B-Diagnostic_procedure
negative: B-Lab_value


In [63]:
predict("Mycoplasma")

1/1 [==============================] - 0s 9ms/step
Predicted Named Entities:
mycoplasma: O


In [21]:
predict("Coronary artery disease (CAD) is a common type of heart disease. It occurs when the arteries that supply blood to the heart muscle become narrowed or blocked due to the buildup of cholesterol and other substances, such as calcium and fatty deposits. This can lead to chest pain (angina), shortness of breath, heart attack, and other complications.")

# predict("Treatment for CAD may include lifestyle changes, such as a healthy diet and exercise, as well as medications to manage cholesterol levels and blood pressure. In some cases, procedures such as angioplasty and stenting may be necessary to open up the blocked arteries and improve blood flow to the heart.")
# People who are at increased risk of developing CAD include those who have a family history of the condition, have high blood pressure, smoke cigarettes, have diabetes, or are overweight or obese. It is important to be aware of the symptoms of CAD and to seek medical attention promptly if you experience chest pain or other signs of heart disease.


1/1 [==============================] - 0s 14ms/step
Predicted Named Entities:
coronary: B-Biological_structure
artery: I-Therapeutic_procedure
disease: I-Disease_disorder
cad: I-Disease_disorder
is: O
a: O
common: O
type: O
of: O
heart: B-Disease_disorder
disease: I-Disease_disorder
it: O
occurs: O
when: O
the: B-Biological_structure
arteries: O
that: I-Coreference
supply: I-Biological_structure
blood: O
to: O
the: B-Biological_structure
heart: I-Biological_structure
muscle: O
become: O
narrowed: B-Sign_symptom
or: O
blocked: O
due: O
to: O
the: O
buildup: O
of: O
cholesterol: O
and: O
other: O
substances: B-Diagnostic_procedure
such: O
as: B-Sign_symptom
calcium: I-Sign_symptom
and: O
fatty: O
deposits: B-Diagnostic_procedure
this: O
can: B-Biological_structure
lead: B-Sign_symptom
to: B-Sign_symptom
chest: I-Sign_symptom
pain: I-Sign_symptom
angina: I-Diagnostic_procedure
shortness: O
of: O
breath: B-Sign_symptom
heart: O
attack: O
and: O
other: O
complications: O


In [65]:
# Define the model architecture
model2 = tf.keras.models.Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=216, input_length=MAX_LENGTH),
    Bidirectional(LSTM(units=64, return_sequences=True)),
    Bidirectional(LSTM(units=32, return_sequences=True)),
    Dense(64, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])


In [66]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 110, 216)          1728000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 110, 128)         143872    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 110, 64)          41216     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 110, 64)           4160      
                                                                 
 dense_3 (Dense)             (None, 110, 85)           5525      
                                                                 
Total params: 1,922,773
Trainable params: 1,922,773
No

In [67]:
# Set the training parameters
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
NUM_EPOCHS = 10

# Train the model
history = model2.fit(train_sequences_padded, train_labels, epochs=10, validation_data=(val_sequences_padded, val_labels))

Epoch 1/10
95/95 [==============================] - 12s 97ms/step - loss: 0.8852 - accuracy: 0.9356 - val_loss: 0.2684 - val_accuracy: 0.9501
Epoch 2/10
95/95 [==============================] - 9s 92ms/step - loss: 0.2639 - accuracy: 0.9466 - val_loss: 0.2379 - val_accuracy: 0.9501
Epoch 3/10
95/95 [==============================] - 9s 93ms/step - loss: 0.2344 - accuracy: 0.9466 - val_loss: 0.2218 - val_accuracy: 0.9504
Epoch 4/10
95/95 [==============================] - 9s 98ms/step - loss: 0.2032 - accuracy: 0.9489 - val_loss: 0.2187 - val_accuracy: 0.9505
Epoch 5/10
95/95 [==============================] - 9s 91ms/step - loss: 0.1784 - accuracy: 0.9524 - val_loss: 0.2201 - val_accuracy: 0.9513
Epoch 6/10
95/95 [==============================] - 9s 93ms/step - loss: 0.1623 - accuracy: 0.9560 - val_loss: 0.2238 - val_accuracy: 0.9526
Epoch 7/10
95/95 [==============================] - 9s 92ms/step - loss: 0.1463 - accuracy: 0.9596 - val_loss: 0.2338 - val_accuracy: 0.9535
Epoch 8/10
9

In [76]:
import re
import string
def tokenize_text(text):
    # Tokenize the text into a list of words
    tokens = []
    for sentence in text.split('\n'):
        for word in sentence.split():
            word = clean_text(word)
            if word.strip():
                tokens.append(word)
    return tokens

def predict(text):
    # tokens = re.findall(r'\b\w+\b', text)
    tokens = tokenize_text(text)
    sequence = tokenizer.texts_to_sequences([' '.join(token for token in tokens)])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post')

    # Make the prediction
    prediction = model2.predict(np.array(padded_sequence))

    # Decode the prediction
    predicted_labels = np.argmax(prediction, axis=-1)
    predicted_labels = [id2label[i] for i in predicted_labels[0]]

    # Print the predicted named entities
    print("Predicted Named Entities:")
    for i in range(len(tokens)):
        print(f"{tokens[i]}: {predicted_labels[i]}")

In [77]:
predict("Coronary artery disease (CAD) is a common type of heart disease. It occurs when the arteries that supply blood to the heart muscle become narrowed or blocked due to the buildup of cholesterol and other substances, such as calcium and fatty deposits. This can lead to chest pain (angina), shortness of breath, heart attack, and other complications.")

1/1 [==============================] - 1s 693ms/step
Predicted Named Entities:
coronary: B-Biological_structure
artery: I-Biological_structure
disease: I-Sign_symptom
cad: O
is: O
a: O
common: O
type: O
of: B-Sign_symptom
heart: I-Sign_symptom
disease: O
it: O
occurs: O
when: O
the: O
arteries: O
that: O
supply: O
blood: O
to: O
the: O
heart: O
muscle: O
become: O
narrowed: O
or: O
blocked: O
due: O
to: B-Medication
the: O
buildup: O
of: O
cholesterol: O
and: O
other: B-Biological_structure
substances: O
such: B-Biological_structure
as: I-Biological_structure
calcium: O
and: O
fatty: O
deposits: O
this: B-Sign_symptom
can: B-Sign_symptom
lead: B-Sign_symptom
to: O
chest: B-Biological_structure
pain: I-Disease_disorder
angina: O
shortness: B-Sign_symptom
of: I-Disease_disorder
breath: O
heart: O
attack: O
and: O
other: O
complications: O


Predicted Named Entities:
coronary: B-Biological_structure
artery: I-Biological_structure
disease: I-Disease_disorder
cad: B-Diagnostic_procedure
common: B-Lab_value
type: B-Diagnostic_procedure
heart: I-Disease_disorder
disease : O
occurs: O
arteries: O
supply: B-Diagnostic_procedure
blood: I-Diagnostic_procedure
heart: O
muscle: B-Sign_symptom
become: O
narrowed: B-Diagnostic_procedure
blocked: I-Sign_symptom
due: B-Diagnostic_procedure
buildup: B-Sign_symptom
cholesterol: I-Sign_symptom
substances: B-Diagnostic_procedure
calcium: B-Biological_structure
fatty: B-Sign_symptom
deposits : B-Sign_symptom
lead: I-Sign_symptom
chest: I-Diagnostic_procedure
pain: O
angina: B-Sign_symptom
shortness: O
breath: O
heart: O
attack: O
complications : O
1/1 [==============================] - 0s 8ms/step
Predicted Named Entities:
treatment: O
cad: O
may: O
include: O
lifestyle: O
changes: O
healthy: O
diet: O
exercise: B-Lab_value
well: B-Diagnostic_procedure
medications: O
manage: O
cholesterol: O
levels: O
blood: O
pressure : B-Sign_symptom
cases: B-Diagnostic_procedure
procedures: O
angioplasty: O
stenting: O
may: B-Sign_symptom
necessary: I-Sign_symptom
open: O
blocked: O
arteries: I-Diagnostic_procedure
improve: O
blood: O
flow: O
heart : O